# Spark Advanced - Spills

## Мотивация

Apache Spark стремится выполнять всю работу в оперативной памяти, но её ресурс ограничен. Если Apache Spark пытается выделить участок памяти, а память уже заполнена, то у Apache Spark есть две стратегии:

- выгрузить часть данных из памяти на диск,
- упасть с `OutOfMemoryError`.

Выгрузить на диск можно лишь закешированные партиции RDD. Дополнительные объекты, которые создаются во время выполнения сортировки, агрегации, перемешивания и т.д. не могут быть выгружены на диск.

Выгрузка RDD на диск называется `spill`. В общении с колегами используют англицизмы: "спил", "спилы".

## Устройство памяти воркера

Вся доступная память разбивается на 4 раздела:

- **зарезервировано**: область памяти, которая используется для внутренних нужд процессов воркера. Всегда равно 300 МБ, не может быть меньше или больше;
- **исполнение (execution)**: область памяти для выполнения логики запросов - сортировка, агрегация, перемешивание и пр.;
- **кеш (storage)**: область памяти доступная для хранения пользовательских объектов в кеше;
- **пользовательские объекты**: область памяти для создания пользовательских объектов (`HashMap`, объекты в `UDF` и т.д).

По умолчанию storage и execution делят пополам оставшуюся память после вычета 300 МБ из всего доступного объема.


![](../imgs/spark-memory-layout.drawio.svg)

### Настройка памяти

При настройке памяти обычно используют два параметра:

- `spark.memory.fraction` указывает размер региона `M` (сумма регионов "исполнение" и "кеш") как процент от значения "Вся_доступная_память - 300M"
  . Значение по умолчанию равно 0.6;
- `spark.memory.storageFraction` указывает размер региона `R` - размер региона в процентах для кеша внутри региона `M`. Значение по умолчанию равно 0.5.

Оставшаяся часть памяти используется:

- для пользовательских объектов (например, переменных в UDF),
- для метаданных Spark,
- как защитный механизм против возникновения `OutOfMemoryError` при редких всплесках нагрузки.

![](../imgs/spark-memory-configs.drawio.svg)

**Внимание**: указанные значения параметров по умолчанию гарантируют приемлимую производительность для большого числа приложений, но глубокий уровень понимания механики управления памятью позволяем получить больший контроль над процессом

### Механизм использования памяти

Границы регионов не являются строгими, Apache Spark может двигать их:

- если приложение не использует кеш, то регион "исполнение" (execution) занимает весь регион `M` (исполнение + кеш);

![](../imgs/spark-memory-no-cache.drawio.svg)
- "кеш" (storage) регион может занять место в регионе "исполнение" (execution), но, если воркеру потребуется память для исполнения, то он начнёт выбрасывать данные из кеша:

![](../imgs/spark-memory-large-cache.drawio.svg)

Выброшенные данные из кеша сохранятся на диске. Этот процесс называется `spill` или спилом. Выгруженные на диск партиции так же иногда называют спилами (spills). Данные, которые хранятся внутри региона `R`, не могут быть выброшены.

> Партиции внутри региона `R` защищены от спилов

#### Регион с пользовательскими объектами

Границы региона с пользовательскими объектами так же могут быть подвинуты исполнением. Это позволяет избежать `OutOfMemoryError`:

![](../imgs/spark-memory-user-objects.drawio.svg)

Но, если необходимо создать пользовательские объекты, то Apache Spark запустит Garbage Collector, чтобы подвинуть границу региона исполнения обратно. Если освободить память будет невозможно, то произойдет `OutOfMemoryError`:

![](../imgs/spark-memory-user-objects-oom.drawio.svg)

## Запуск приложения

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark import StorageLevel

### Создание сессии

In [ ]:
spark = (
    SparkSession
        .builder
        .appName("Spills")
        .master("local[4]")
        .config("spark.memory.fraction", 0.6)
        .config("spark.memory.storageFraction", 0.5)
        .getOrCreate()
)
sc = spark.sparkContext

Spark UI доступен на порту [4040](http://localhost:4040)

### Подготовка данных

In [ ]:
!mkdir -p /tmp/taxi
!unzip -o -d /tmp/taxi ./data/taxi.zip

In [ ]:
sc.setJobDescription("Разбить датафрейм на 4 партиции")
spark.read.parquet("/tmp/taxi") \
    .repartition(4) \
    .write.mode("overwrite").save("/tmp/taxi_many")

### Загрузка основного датафрейма

In [ ]:
def load_taxi_df(sparkContext):
    sparkContext.setJobDescription("Загрузить датафрейм с 4 партициями")
    return spark.read.parquet("/tmp/taxi_many")

In [ ]:
taxi_df = load_taxi_df(sc)

In [ ]:
taxi_df.printSchema()

In [ ]:
taxi_df.rdd.getNumPartitions()

In [ ]:
sc.setJobDescription("Распределение данных по партициям")
taxi_df.groupby(F.spark_partition_id()).count().show()

### Доступные ресурсы

In [ ]:
from typing import NamedTuple

class MemoryDescription(NamedTuple):
    total_memory: float = spark._jvm.java.lang.Runtime.getRuntime().maxMemory() / 1024 / 1024
    reserved_memory: int = 300
    available_memory: float = total_memory - reserved_memory

    memoryFraction: float = float(spark.conf.get("spark.memory.fraction"))
    storageFraction: float = float(spark.conf.get("spark.memory.storageFraction"))

    user_objects: float = available_memory * (1 - memoryFraction)
    
    m_region: float = available_memory * memoryFraction
    execution: float = m_region * (1 - storageFraction)
    storage: float = m_region * storageFraction

description = MemoryDescription()

print(f"Объем памяти воркера: {description.total_memory} MB")
print(f"Объем зарезервированной памяти: 300 MB")
print(f"Объем доступной памяти: {description.available_memory} MB")
print(f"Объем пользовательских данных: {description.user_objects} MB")
print(f"Объем региона M: {description.m_region} MB")
print(f"Объем региона исполнения: {description.execution} MB")
print(f"Объем региона кеширования `R`: {description.storage} MB")

Объем доступной памяти можно посмотреть на Spark UI на вкладке [Executors](http://localhost:4040/executors/)

![](../imgs/spark-available-memory.drawio.svg)

## Исполнение вытесняет кеш

### Занять кеш

Регион `R`, в границах которого закешированные партиции невозможно вытеснить, имеет размер:

In [ ]:
print(f"{MemoryDescription().storage}M")

Но кеш в Apache Spark может занимать весь регион M, размер которого:

In [ ]:
print(f"{MemoryDescription().m_region}M")

Следущий код демонстрирует факт, что объем закешированных данных может превышать границы региона `R`:

In [ ]:
def fill_cache(taxi_df):
    sc.setJobDescription("Занять кеш `0 == 0`")

    cached_taxi_df = taxi_df.where("0 == 0").persist(StorageLevel.MEMORY_ONLY)
    cached_taxi_df.count()

    sc.setJobDescription("Занять кеш `1 == 1`")
    cached_taxi_11_df = taxi_df.where("1 == 1").persist(StorageLevel.MEMORY_ONLY)
    cached_taxi_11_df.count()

In [ ]:
fill_cache(taxi_df)

In [ ]:
def calculate_cache_size(spark_context):
    return sum([x.memSize() for x in spark_context._jsc.sc().getRDDStorageInfo()]) / 1024 / 1024    

In [ ]:
total_cache_occupied = calculate_cache_size(sc)
print(f"Объем кеша в памяти {total_cache_occupied:.2f}M, что превышает {MemoryDescription().storage}M - размер R региона")

Размер датафрейма так же можно увидеть и в [Spark UI](http://localhost:4040/storage/)

![](../imgs/spark-full-taxi-df-cache.drawio.svg)

### Посчитать уникальные строки

Подсчет уникальных строк потребует загрузки датафрейма в регион "исполнение" и создание дополнительных объектов (`HashMap`) для получения результата. При нехватке памяти:

- часть партиций будет вытеснена из кеша,
- также загруженные партиции из региона "исполнение" могут несколько раз выгрузиться из памяти на диск, а потом вернуться обратно в память,
- только служебные структуры данных (`HashMap`) не могут быть выгружены на диск.

In [ ]:
def unique_rows_count(df):
    sc.setJobDescription("Вычислить уникальное число строк")
    
    return df.distinct().count()

In [ ]:
unique_rows_count(taxi_df)

В результате выполнения операции часть данных в кеше пропала из памяти:

In [ ]:
total_cache_occupied = calculate_cache_size(sc)
print(f"Объем кеша в памяти {total_cache_occupied:.2f}M, а размер региона R {MemoryDescription().storage}M")

Объем данных в кеше уменьшился, одна партиция была выгружена из кеша:

![](../imgs/spark-cache_spills_05.drawio.svg)

Общий объем спилов (spills) можно посмотреть на странице стадии:


![](../imgs/spark-stage-spill-header.drawio.svg)

На картинке выше можно увидеть две новые метрики:

- **Spill (Memory)** - объем данных, который был вытеснен из памяти,
- **Spill (Disk)** - объем данных, который был сохранен на диске в сжатом виде.

Можно отметить разницу в объеме одних и те же данных при хранении на диске и в памяти.

Информация в заголовке показывает общие числа на все задания (tasks). На странице стадии так же можно найти статистику по спилам в виде гистограммы с персентилями:

![](../imgs/spark-stage-spill-stat-05.drawio.svg)

А так же информация по спилам по каждому заданию:

![](../imgs/spark-stage-spill-tasks-05.drawio.svg)

Для сравнения необходимо изменить границу R региона, а для этого необходимо полностью остановить приложение и запустить его с новыми настройками границ:

In [ ]:
spark.stop()

### Установка новой границы кеша

В качестве новой границы региона кеша (`R`) выберем значение `0.1`:

In [ ]:
spark = (
    SparkSession
        .builder
        .appName("Spills")
        .master("local[4]")
        .config("spark.memory.fraction", 0.6)
        .config("spark.memory.storageFraction", 0.1) # новая грацниа кеша
        .getOrCreate()
)

sc = spark.sparkContext

Необходимо явно запустить GC для очистки мусора:

In [ ]:
spark._jvm.java.lang.System.gc()

### Загрузка основного датафрейма

In [ ]:
taxi_df = load_taxi_df(sc)

### Занять кеш

In [ ]:
print(f"Граница региона (`R`) кеша: {MemoryDescription().storage:.2f}M")

In [ ]:
fill_cache(taxi_df)

In [ ]:
total_cache_occupied = calculate_cache_size(sc)
print(f"Объем кеша в памяти {total_cache_occupied:.2f}M, что превышает {MemoryDescription().storage}M - размер R региона")

Размер кешей так же можно увидеть и в [Spark UI](http://localhost:4040/storage/):
- [Storage](http://localhost:4040/storage/)
- [Executors](http://localhost:4040/executors/)

![](../imgs/spark-full-taxi-df-cache.drawio.svg)

In [ ]:
unique_rows_count(taxi_df)

In [ ]:
total_cache_occupied = calculate_cache_size(sc)
print(f"Объем кеша в памяти {total_cache_occupied:.2f}M из {MemoryDescription().storage}M доступных")

Можно заметить, что в этот раз вытеснилось больше партиций из кеша:

![](../imgs/spark-cache_spills_01.drawio.svg)

А также можно увидеть, что больше спилов происходило во время работы задачи:

![](../imgs/spark-stage-spill-header-01.drawio.svg)

### Способы избежать спилы

1. добавить оперативной памяти на воркеры;
1. из памяти могут вытесняться только партиции целиком, следовательно, вытеснение больших партиции освободит больше места, но иногда необходимо освободить немного места, поэтому предпочтительно работать с небольшими партициями. Настройка следующих опций позволит управлять размером партиций:
    - `spark.sql.shuffle.partitions` - сколько партиций будет создаваться после shuffle. Выше значение - меньший размер партиций;
    - `spark.sql.files.maxPartitionBytes` - сколько байт в одной партиции: даже если на диске ваши parquet файлы хранятся очень гранулировано, Spark будет объединять их, чтобы получить более крупные партиции. Много небольших партиций могут объединиться в одну большую, но по размеру не превышающую значение указанное в этой настройке.
1. явный вызов [`DataFrame#repartition`](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.repartition.html) для разбиения партиций на более мелкие.

### Вывод

Apache Spark старается выполнять всю работу в памяти, но ее ресурс ограничен, поэтому существует механизм спилов: хранение части данных на диске. Спилами могут быть только партиции RDD. Учитывая повсеместное распространение SSD дисков, спилы могут и не быть на столько большой проблемой, какой она представлялась десять лет назад. Но не смотря на это, нужно учитывать, что случайное чтение с SSD диска всё еще в 170 раз медленее чтения из памяти, а поэтому для тонкой настройки нужно понимать почему спилы происходят и как бороться с ними. Понимание устройства памяти воркеров, и как она распределяется, позволит тонко настроить ее под любое приложение.

### Задание

1. Разбить `taxi_df` на 100 партиций и сохранить на диск в директорию `/tmp/taxi_100`;
1. Загрузить новый датафрейм `taxi_100_df` из `/tmp/taxi_100`;
1. Вызывать `fill_cache` для заполнения кеша;
1. Сколько памяти занято кешем?
1. Запустить `unique_rows_count(taxi_100_df)`, убедиться, что в работе участвуют 100 патиций
<details>
    <summary>Подсказка</summary>

    Необходимо пограть с настройкой `spark.sql.files.maxPartitionBytes`
</details>

6. Установить границу `M` региона в значение `0.8`, запустить шаги 1-5 снова. Что произошло?
<details>
    <summary>Подсказка</summary>

    Остановить Spark приложение и запустить снова с новыми границами. Не забудьте вызывать <code>spark._jvm.java.lang.System.gc()</code>
</details>

7. Установить границу `M` региона в значение `0.3`, запустить шаги 1-5 снова. Что произошло?
<details>
    <summary>Подсказка</summary>

    Остановить Spark приложение и запустить снова с новыми границами. Не забудьте вызывать <code>spark._jvm.java.lang.System.gc()</code>
</details>

8. Установить границу `R` региона в значение `0.7`, запустить шаги 1-5 снова. Что произошло?
<details>
    <summary>Подсказка</summary>

    Остановить Spark приложение и запустить снова с новыми границами. Не забудьте вызывать <code>spark._jvm.java.lang.System.gc()</code>
</details>

9. Какой вывод можно сделать относительно значениий для региона `M` по умолчанию?
<details>
    <summary>Подсказка</summary>

    Значения по умолчанию на удивление позволяют работать без OOM
</details>
